In [35]:
import requests
import openpyxl

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 네이버 쇼핑 사이트 가져오기
driver.get("https://shopping.naver.com")
# 검색어 입력받기
product = input("검색할 상품을 입력하세요")
# 검색어 자동화 입력
click_=driver.find_element(By.CLASS_NAME,'_combineHeader_expansion_search_inner_1VxB3')
click_.click()
sleep(1)
search = driver.find_element(By.CLASS_NAME, '_ac_input._searchInput_input_text_2Jhbj')
search.send_keys(product)
sleep(1)
enter = driver.find_element(By.CLASS_NAME, '_combineHeader_button_2xfGa')
enter.click()
# 엑셀 만들기
wb = openpyxl.Workbook()
# 워크시트 만들기
ws = wb.active
ws.title = product
# 데이터 추가하기
ws.append(['상품명', '판매가', 'url'])

# 초기 스크롤 높이
last_height = driver.execute_script("return Math.max( document.body.scrollHeight, document.body.offsetHeight, document.documentElement.clientHeight, document.documentElement.scrollHeight, document.documentElement.offsetHeight);")
# 스크롤을 끝까지 내리면서 데이터 수집
while True:
  # 상품 클래스 선택
  product_elements = driver.find_elements(By.CLASS_NAME,'adProduct_item_inner__ObdW2')
  # 클래스 선택해서 값을 가져온 뒤 엑셀에 삽입
  for element in product_elements:
    title_elements = element.find_element(By.CLASS_NAME,'adProduct_info_tit__yhsNL')
    title = title_elements.text
    price_elements = element.find_element(By.CLASS_NAME,'adProduct_num__wt79O')
    price = price_elements.text
    url_element = element.find_element(By.CLASS_NAME,'adProduct_btn_link__ufG7R ')
    url = url_element.get_attribute('href')
    # 엑셀에 삽입
    ws.append([title,price,url])

  # 스크롤 다운
  driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
  sleep(2)
  # 새로운 스크롤 높이 계산
  new_height = driver.execute_script("return Math.max( document.body.scrollHeight, document.body.offsetHeight, document.documentElement.clientHeight, document.documentElement.scrollHeight, document.documentElement.offsetHeight);")
  # 스크롤이 끝에 도달하면 반복문 종료
  if new_height == last_height:
    break
  # 스크롤 높이 업데이트
  last_height = new_height

# 엑셀 저장하기
wb.save(f'./gmarket_{product}.xlsx')